%md
### Impacto do aumento da Taxa de Juros na Carteira de Crédito dos bancos

In [0]:
from pyspark.sql import SparkSession
import requests

spark.sql('CREATE DATABASE IF NOT EXISTS dlt_bacen;') 

In [0]:
# TAXA DE JUROS ACUMULADA (SELIC)

codigo = '1178'
api = 'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo)

spark = SparkSession.builder.appName('Bacen').getOrCreate()
requisicao = requests.get(api)

data = requisicao.json()
df = spark.createDataFrame(data)

df.createOrReplaceTempView('selic')

query = spark.sql("""
WITH DataIdeal AS (
    SELECT
        DATE_TRUNC('month', TO_DATE(data, 'dd/MM/yyyy')) AS MesAno,
        MIN(TO_DATE(data, 'dd/MM/yyyy')) AS DataAntiga
    FROM selic
    WHERE TO_DATE(data, 'dd/MM/yyyy') >= '2000-03-01'
    GROUP BY DATE_TRUNC('month', TO_DATE(data, 'dd/MM/yyyy'))
)
SELECT
    a.DataAntiga AS Mothn_year,
    TRY_CAST(REPLACE(b.valor, ',', '.') AS DECIMAL(10, 2)) AS Tx_Selic
FROM DataIdeal a
JOIN selic b ON TO_DATE(b.data, 'dd/MM/yyyy') = a.DataAntiga
ORDER BY a.DataAntiga;
""")

query.write.mode('overwrite').saveAsTable('dlt_bacen.tb_selic_history')

query.display()

In [0]:
# TAXA DE INADIMPLÊNCIA

codigo = '13673'
api = 'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo)

spark = SparkSession.builder.appName('Bacen').getOrCreate()
requisicao = requests.get(api)

data = requisicao.json()
df = spark.createDataFrame(data)

df.createOrReplaceTempView('inadimplencia')

query = spark.sql("""
SELECT
    TO_DATE(data, 'dd/MM/yyyy') AS Month_year,
    TRY_CAST(REPLACE(valor, ',', '.') AS DECIMAL(10, 2)) AS Tx_Inad
FROM inadimplencia;
""")                       

query.write.mode('overwrite').saveAsTable('dlt_bacen.tb_inad_history')

query.display()

In [0]:
# TAXA MÉDIA DE JUROS DE CRÉDITO 

codigo = '20714'
api = 'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo)

spark = SparkSession.builder.appName('Bacen').getOrCreate()
requisicao = requests.get(api)

data = requisicao.json()
df = spark.createDataFrame(data)

df.createOrReplaceTempView('credito')

query = spark.sql("""
SELECT
    TO_DATE(data, 'dd/MM/yyyy') AS Mothn_year,
    TRY_CAST(REPLACE(valor, ',', '.') AS DECIMAL(10, 2)) AS Tx_credit
FROM credito
""")

query.write.mode('overwrite').saveAsTable('dlt_bacen.tb_credit_history')

query.display()

In [0]:
analyze = spark.sql("""
SELECT
  a.Mothn_year,
  a.Tx_credit,
  b.Tx_Inad
FROM
  dlt_bacen.tb_credit_history a
  LEFT JOIN dlt_bacen.tb_inad_history b ON a.Mothn_year = b.Month_year
ORDER BY
  a.Mothn_year 
""")

analyze.display()

In [0]:
import pandas as pd
import matplotlib.pyplot as plt

analyze_pd = analyze.toPandas()

plt.figure(figsize=(12, 6))
plt.plot(analyze_pd['Mothn_year'], analyze_pd['Tx_credit'], label='Taxa de Crédito')
plt.plot(analyze_pd['Mothn_year'], analyze_pd['Tx_Inad'], label='Taxa de Inadimplência')

plt.xlabel('Mês e Ano')
plt.ylabel('Taxa')
plt.title('Evolução das Taxas ao Longo do Tempo')
plt.legend()
plt.grid(True)
plt.show()